# SparkML Models Notebook

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.sql.types import DoubleType

In [2]:
spark = SparkSession.builder.appName("ProjectSuccessPred").getOrCreate()

24/05/20 22:56:13 WARN Utils: Your hostname, ferys-laptop resolves to a loopback address: 127.0.1.1; using 192.168.1.100 instead (on interface wlo1)
24/05/20 22:56:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/20 22:56:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load data
data = spark.read.csv("kick_starter_cleaned.csv", header=True, inferSchema=True)

# Drop specified columns
columns_to_drop = ["category", "currency", "goal", "deadline", "launched", "pledged", "backers"]
data = data.drop(*columns_to_drop)

# Examine and transform the 'state' column
data = data.withColumn("label", when(col("state") == "successful", 1).otherwise(0))

# Drop rows where the state is 'live'
data = data.filter(col("state") != "live")

# Drop the 'state' column as it's now redundant
data = data.drop("state")

In [4]:
# Columns to index and encode
categorical_columns = ["name", "main_category", "country"]

# Indexing and encoding pipelines
indexers = [StringIndexer(inputCol=col, outputCol=col+"_indexed") for col in categorical_columns]
encoders = [OneHotEncoder(inputCol=col+"_indexed", outputCol=col+"_encoded") for col in categorical_columns]

# Apply the indexers and encoders
pipeline = Pipeline(stages=indexers + encoders)
data = pipeline.fit(data).transform(data)

# Drop the original and indexed columns, keeping only encoded columns
data = data.drop(*categorical_columns)
for col in categorical_columns:
    data = data.drop(col+"_indexed")

In [5]:
# Convert string columns to double type
data = data.withColumn("usd_pledged_real", data["usd_pledged_real"].cast(DoubleType()))
data = data.withColumn("usd_goal_real", data["usd_goal_real"].cast(DoubleType()))

# Define feature columns (excluding 'label')
feature_columns = [col for col in data.columns if col != "label"]

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(data)


# Select the relevant columns for modeling
data = data.select("features", "label")

In [6]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=1234)

In [7]:
lr = LogisticRegression(featuresCol="features", labelCol="label")
lr_model = lr.fit(train_data)
lr_predictions = lr_model.transform(test_data)

24/05/20 22:56:33 WARN DAGScheduler: Broadcasting large task binary with size 56.6 MiB
24/05/20 22:56:37 ERROR Executor: Exception in task 1.0 in stage 11.0 (TID 53)2]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinary

Py4JJavaError: An error occurred while calling o422.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 11.0 failed 1 times, most recent failure: Lost task 1.0 in stage 11.0 (TID 53) (192.168.1.100 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.reflect.Array.newInstance(Array.java:78)
	at java.base/java.io.ObjectInputStream.readArray(ObjectInputStream.java:2098)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1675)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerialData(ObjectInputStream.java:2390)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2228)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.defaultReadFields(ObjectInputStream.java:2496)
	at java.base/java.io.ObjectInputStream.readSerial

24/05/20 22:56:41 ERROR Utils: Uncaught exception in thread Executor task launch worker for task 11.0 in stage 11.0 (TID 63)
java.lang.NullPointerException
	at org.apache.spark.scheduler.Task.$anonfun$run$3(Task.scala:147)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.Task.run(Task.scala:144)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:94)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:623)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
24/05/20

In [ ]:
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label")
dt_model = dt.fit(train_data)
dt_predictions = dt_model.transform(test_data)

In [ ]:
rf = RandomForestClassifier(featuresCol="features", labelCol="label")
rf_model = rf.fit(train_data)
rf_predictions = rf_model.transform(test_data)